In [ ]:
pip install boto3

In [ ]:
from google.colab import drive
drive.mount("/content/drive/")

In [ ]:
import boto3
import json
import csv

In [ ]:
def detect_faces(photo,bucket,session,user_no):
  client=session.client('rekognition',region_name='ap-south-1')
  response = client.detect_faces(Image={'S3Object':{'Bucket':bucket,'Name':photo}},Attributes=['ALL'])
  print('Detected faces for ' + photo)
  face_count=0
  for faceDetail in response['FaceDetails']:
     face_count+=1
     # write feature only for first face
     if(face_count>1):
       continue

     dict_data=[faceDetail]
     csv_columns = ['FileName','BoundingBox','AgeRange','Smile','Eyeglasses','Sunglasses','Gender','Beard','Mustache','EyesOpen','MouthOpen','Emotions','Landmarks','Pose','Quality','Confidence']
     # emotion dict
     emotion_conf={"CALM":0,"ANGRY":0,"CONFUSED":0,"SURPRISED":0,"DISGUSTED":0,"SAD":0,"FEAR":0,"HAPPY":0}
     # get the name of photo 
     photo_name=photo.split("/")[-1]
     
     actual_photo="user_"+str(user_no)+"/"+photo_name
     filename="/content/drive/MyDrive/InputFolder/NewData/Output1/user_"+str(user_no)+".csv"
     try:
       with open(filename,'a') as csvfile:
         writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
         for data in dict_data:
           print(data)
           data['FileName']=actual_photo
           # take the actual emotions 
           row=data["Emotions"]
           # reorder emotions
           for j in row:
             emotion_conf[j['Type']]=j['Confidence']
           # change in data  
           data["Emotions"]=emotion_conf  
           writer.writerow(data)
     except IOError:
       print("I/O error")      

  return len(response['FaceDetails'])




def main():

  # give user number
  user_no=12
  # folder name in S3 bucket
  folder_name="new_data"
  # specify directory structure of file in S3 bucket
  prefix=folder_name+"/user_"+str(user_no)+"/"+"saved_images3/"
   # S3 bucket name
  BUCKET_NAME="facial-images"
  # provide credentials
  # provide aws access key_id and aws secret access key
  session=boto3.Session(aws_access_key_id='key id',aws_secret_access_key='access key')
  s3=session.client('s3')

  # prepare file for writting features
  csv_columns = ['FileName','BoundingBox','AgeRange','Smile','Eyeglasses','Sunglasses','Gender','Beard','Mustache','EyesOpen','MouthOpen','Emotions','Landmarks','Pose','Quality','Confidence']
  filename="/content/drive/MyDrive/InputFolder/NewData/Output1/user_"+str(user_no)+".csv"
  
  
  try:
      with open(filename, 'w') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
            writer.writeheader()
  except IOError:
            print("I/O error")
  
  # list all the files in s3 bucket
  file_list = s3.list_objects_v2(Bucket=BUCKET_NAME,Prefix=prefix,Delimiter='/')
  # process each file
  for i in (file_list['Contents']):
    if(".jpg" in i['Key']):
      photo=i['Key']
      #print(photo)

      face_count=detect_faces(photo,BUCKET_NAME,session,user_no)
      print(face_count)

# driver code
main()      

Detected faces for new_data/user_23/saved_images3/IMG_20210814_105813.jpg
{'BoundingBox': {'Width': 0.7016516327857971, 'Height': 0.5932685732841492, 'Left': 0.0800691768527031, 'Top': 0.41918638348579407}, 'AgeRange': {'Low': 30, 'High': 40}, 'Smile': {'Value': False, 'Confidence': 92.61739349365234}, 'Eyeglasses': {'Value': True, 'Confidence': 53.524227142333984}, 'Sunglasses': {'Value': False, 'Confidence': 93.30294036865234}, 'Gender': {'Value': 'Female', 'Confidence': 99.7641372680664}, 'Beard': {'Value': False, 'Confidence': 81.43232727050781}, 'Mustache': {'Value': False, 'Confidence': 92.9434814453125}, 'EyesOpen': {'Value': True, 'Confidence': 91.14518737792969}, 'MouthOpen': {'Value': False, 'Confidence': 79.40757751464844}, 'Emotions': [{'Type': 'SAD', 'Confidence': 98.97599792480469}, {'Type': 'FEAR', 'Confidence': 0.42033135890960693}, {'Type': 'CALM', 'Confidence': 0.3078760504722595}, {'Type': 'DISGUSTED', 'Confidence': 0.07879777252674103}, {'Type': 'ANGRY', 'Confidence